In [4]:
import pandas as pd
import pickle
import os
from tqdm import tqdm

# 📌 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 📌 Cargar archivos`

ruta_parquet = "/content/drive/MyDrive/Datos/2_2_BaseInicial.parquet"
ruta_archivos_leidos = "/content/drive/MyDrive/Datos/1_ArchivosLeidos.pkl"

base_inicial = pd.read_parquet(ruta_parquet)



with open(ruta_archivos_leidos, "rb") as f:
    dataframes = pickle.load(f)



### 🔹 Paso 3: Calcular `RQ` (Riesgo de Quiebra)
print("\n🔄 Verificando continuidad en el envío de estados financieros...")

# Asignar `RQ = 0` a todas las empresas por defecto
base_inicial["RQ"] = 0

# Crear una copia de `base_inicial` desplazada un año adelante
base_siguiente = base_inicial[['NIT', 'Año']].copy()
base_siguiente['Año'] -= 1  # Restamos 1 año para hacer el merge con el año anterior

# Fusionar para ver si el NIT del año actual tiene datos en el año siguiente
base_inicial = base_inicial.merge(base_siguiente, on=['NIT', 'Año'], how='left', indicator=True)

# Si el indicador (`_merge`) dice que no tiene coincidencia en el año siguiente, `RQ = 1`
base_inicial.loc[(base_inicial['_merge'] == 'left_only') & (base_inicial['Año'] != 2023), 'RQ'] = 1

# Eliminar la columna temporal `_merge`
base_inicial.drop(columns=['_merge'], inplace=True)

print("\n🔄 Verificando 'Estado actual' en empresas 2016–2023...")

# Crear un DataFrame vacío para almacenar los estados de empresas 2016–2023
estado_empresas = pd.DataFrame(columns=['NIT', 'Año', 'Estado actual'])

# Recorrer los años 2016–2023 y combinar los estados de Pymes y Plenas
for year in range(2016, 2024):
    for tipo, sufijo in [("Pymes", "_PY_CA"), ("Plenas", "_PL_CA")]:
        key = f"{year}{sufijo}"
        if key in dataframes:
            df_temp = dataframes[key][['NIT', 'Estado actual']].copy()
            df_temp['Año'] = year
            estado_empresas = pd.concat([estado_empresas, df_temp], ignore_index=True)

# Fusionar `base_inicial` con `estado_empresas` para asignar el "Estado actual"
base_inicial = base_inicial.merge(estado_empresas, on=['NIT', 'Año'], how='left')

# Si "Estado actual" no es ACTIVA o EN ETAPA PREOPERATIVA, poner `RQ = 1`
base_inicial.loc[(base_inicial['Estado actual'].notna()) &
                 (~base_inicial['Estado actual'].isin(['ACTIVA', 'EN ETAPA PREOPERATIVA'])), 'RQ'] = 1

# Eliminar la columna "Estado actual"
base_inicial.drop(columns=['Estado actual'], inplace=True)

# 📌 Estadísticas generales
total_registros = len(base_inicial)
empresas_totales = base_inicial['NIT'].nunique()

print(f"📊 Total de registros empresa-año (unidades de estudio): {total_registros}")
print(f"🏢 Total de empresas únicas (NIT distintos): {empresas_totales}")

# 📌 Conteo de registros según RQ
total_rq_1 = (base_inicial['RQ'] == 1).sum()
total_rq_0 = (base_inicial['RQ'] == 0).sum()

print(f"\n⚠️ Registros con riesgo de quiebra (RQ = 1): {total_rq_1}")
print(f"✅ Registros sin riesgo de quiebra (RQ = 0): {total_rq_0}")

# 📌 Empresas con al menos un RQ = 1
empresas_con_riesgo = base_inicial[base_inicial['RQ'] == 1]['NIT'].nunique()
print(f"\n🚨 Empresas que tuvieron al menos un año con riesgo de quiebra (RQ = 1): {empresas_con_riesgo}")

# 📌 Empresas que nunca presentaron riesgo (solo RQ = 0)
nits_rq_0 = set(base_inicial[base_inicial['RQ'] == 0]['NIT'])
nits_rq_1 = set(base_inicial[base_inicial['RQ'] == 1]['NIT'])
empresas_siempre_sanas = len(nits_rq_0 - nits_rq_1)

print(f"🟢 Empresas que siempre estuvieron sin riesgo (RQ = 0 en todos los años): {empresas_siempre_sanas}")

# 📌 Paso final: Guardar la base con nombre correcto
print("\n💾 Guardando la base de datos...")

# Definir ruta base (sin extensión)
ruta_base = "/content/drive/MyDrive/Datos/3_BaseInicial_RQ"

# Crear carpeta si no existe
os.makedirs(os.path.dirname(ruta_base), exist_ok=True)

# Guardar en formato Pickle (.pkl)
with open(ruta_base + ".pkl", "wb") as f:
    pickle.dump(base_inicial, f)
print(f"✅ Guardado en formato Pickle: {ruta_base}.pkl")

# Guardar en formato Parquet (.parquet)
base_inicial.to_parquet(ruta_base + ".parquet", index=False)
print(f"✅ Guardado en formato Parquet: {ruta_base}.parquet")

# 📊 Variables auxiliares para el resumen
columnas = list(base_inicial.columns)
total_columnas = len(columnas)
ejemplo_nits = base_inicial['NIT'].dropna().astype(str).unique()[:10]
ejemplo_años = sorted(base_inicial['Año'].dropna().unique())[:10]

# 📊 Preparar elementos del resumen
columnas = list(base_inicial.columns)
total_columnas = len(columnas)
ejemplo_nits = base_inicial['NIT'].dropna().astype(str).unique()[:10]
ejemplo_años = sorted(base_inicial['Año'].dropna().unique())[:10]

# 📄 Crear resumen del paso 3
resumen_txt = f"""
📄 Script: 3_Creacion_Variable_RQ.ipynb
📅 Fecha: {pd.Timestamp.today().strftime('%Y-%m-%d')}
🧩 Objetivo: Construir la variable binaria `RQ` que indica riesgo de quiebra por no continuidad o estado administrativo.

🔁 Pasos realizados:
- Se asigna RQ = 1 si no hay continuidad al año siguiente (salvo en 2023)
- Se asigna RQ = 1 si el "Estado actual" no es ACTIVA o EN ETAPA PREOPERATIVA entre 2016-2023

📦 Archivos generados:
- 3_BaseInicial_RQ.pkl
- 3_BaseInicial_RQ.parquet

📊 Estadísticas:
- Registros totales: {len(base_inicial)}
- NITs únicos: {base_inicial['NIT'].nunique()}
- RQ = 1 (riesgo): {(base_inicial['RQ'] == 1).sum()}
- RQ = 0 (sin riesgo): {(base_inicial['RQ'] == 0).sum()}
- Empresas que alguna vez tuvieron RQ = 1: {base_inicial[base_inicial['RQ'] == 1]['NIT'].nunique()}
- Empresas siempre sin riesgo: {len(set(base_inicial[base_inicial['RQ'] == 0]['NIT']) - set(base_inicial[base_inicial['RQ'] == 1]['NIT']))}

📄 Columnas disponibles: {total_columnas}
🆔 Ejemplos de NITs: {', '.join(ejemplo_nits)}
📅 Años presentes: {', '.join(map(str, ejemplo_años))}
"""

# 📋 Agregar nombres de columnas al resumen
columnas_ordenadas = "\n".join([f"   • {col}" for col in columnas])
resumen_txt += f"""

📋 Nombres de columnas:
{columnas_ordenadas}
"""

# 📄 Guardar archivo .txt
ruta_resumen = "/content/drive/MyDrive/Datos/3_resumen.txt"
with open(ruta_resumen, "w", encoding="utf-8") as f:
    f.write(resumen_txt.strip())

print(f"📝 Resumen guardado en: {ruta_resumen}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

🔄 Verificando continuidad en el envío de estados financieros...

🔄 Verificando 'Estado actual' en empresas 2016–2023...
📊 Total de registros empresa-año (unidades de estudio): 542083
🏢 Total de empresas únicas (NIT distintos): 69423

⚠️ Registros con riesgo de quiebra (RQ = 1): 71973
✅ Registros sin riesgo de quiebra (RQ = 0): 470110

🚨 Empresas que tuvieron al menos un año con riesgo de quiebra (RQ = 1): 51275
🟢 Empresas que siempre estuvieron sin riesgo (RQ = 0 en todos los años): 18148

💾 Guardando la base de datos...
✅ Guardado en formato Pickle: /content/drive/MyDrive/Datos/3_BaseInicial_RQ.pkl
✅ Guardado en formato Parquet: /content/drive/MyDrive/Datos/3_BaseInicial_RQ.parquet
📝 Resumen guardado en: /content/drive/MyDrive/Datos/3_resumen.txt
